## Description
What I did previously, is I downloaded 10 projects the is developed by Chinese-speaking programmers, and 10 projects that is developed by English-speaking programmers, all the chinese projects are stored under the directory:
```
RetrieveDataProject/DownloadProjects/ChineseProjects
```
And all the english projects are stored under the directory:
```
RetrieveDataProject/DownloadProjects/EnglishProjects
```
In this file, I will read from those python files, and for each python file, I will parse them to get all the variable and function names.

In [24]:
import ast
import os

import pandas as pd
import ast_scope


This following function will open the python file under parameter "file_path", and parse the file to get variable names and their scope: FunctionScope or GlobalScope. 

In [25]:
index_counter = 0
def extract_variables_and_functions_naive(file_path, author_type, project_name):
    global index_counter
    try:
        with open(file_path, 'r') as file:
            tree = ast.parse(file.read(), filename=file_path)
    except:
        # If a syntax error occurs, print the error message
        print(f"Syntax error in file '{file_path}'")
        raise
    scope_info = ast_scope.annotate(tree)
    names = []
    # todo: function parameter, other?
    for node in ast.walk(tree):
        if isinstance(node, ast.FunctionDef):
            nameScope = "GlobalScope" if isinstance(scope_info[node], ast_scope.scope.GlobalScope) else "FunctionScope"
            name = {"id": index_counter, "name": node.name, "nameType": "function", "nameScope": nameScope,
                    "author": author_type, "project_name": project_name}
            names.append(name)
            index_counter += 1
        elif isinstance(node, ast.Assign):
            for target in node.targets:
                if isinstance(target, ast.Name):
                    nameScope = "GlobalScope" if isinstance(scope_info[target],
                                                            ast_scope.scope.GlobalScope) else "FunctionScope"
                    name = {"id": index_counter, "name": target.id, "nameType": "variable", "nameScope": nameScope,
                            "author": author_type, "project_name": project_name}
                    names.append(name)
                    index_counter += 1
        elif isinstance(node, ast.ClassDef):
            nameScope = "GlobalScope" if isinstance(scope_info[node], ast_scope.scope.GlobalScope) else "FunctionScope"
            name = {"id": index_counter, "name": node.name, "nameType": "class", "nameScope": nameScope,
                    "author": author_type, "project_name": project_name}
            names.append(name)
            index_counter += 1
    # Now write the result to pandas dataframe
    names_df = pd.DataFrame.from_records(names)
    return names_df


The following function will walk through a project directory, and open any python files, and parse them. 

In [26]:
def parse_all_python_files_in_project(project_path, project_name, author_type, db_conn, db_table):
    for foldername, subfolders, filenames in os.walk(project_path):
        for filename in filenames:
            if fnmatch.fnmatch(filename, '*.py'):
                filepath = os.path.join(foldername, filename)
                try:
                    names_df = extract_variables_and_functions_naive(filepath, author_type=author_type, project_name=project_name)
                    names_df.to_sql(db_table, db_conn, if_exists='append', index=False)
                except:
                    continue
                

Now we will open a database to save our result. We will place the database under the same directory of this notebook. 

In [27]:
import sqlite3
conn = sqlite3.connect('data.db')
table = 'NameTable'

We won't run this notebook often, but if we run this notebook, we will reset the table. 

In [29]:
# Create a cursor object
cursor = conn.cursor()
# SQL statement to drop the table
sql_statement = f"DROP TABLE IF EXISTS {table}"
cursor.execute(sql_statement)
conn.commit()

Then we will parse all the python files written by Chinese-speaking programmers. 

In [31]:
import os
import fnmatch
import shutil

chinese_root_directory = '../RetrieveDataProject/DownloadProjects/ChineseProjects'
for project_name in os.listdir(chinese_root_directory):
    project_path = os.path.join(chinese_root_directory, project_name)
    # walk through project path
    parse_all_python_files_in_project(project_path, project_name, author_type='Chinese', db_conn=conn, db_table=table)

../RetrieveDataProject/DownloadProjects/ChineseProjects/GPT-SoVITS/webui.py:109: SyntaxWarning: invalid escape sequence '\d'
  parts = re.split('(\d+)', s)
../RetrieveDataProject/DownloadProjects/ChineseProjects/GPT-SoVITS/GPT_SoVITS/inference_webui.py:520: SyntaxWarning: invalid escape sequence '\d'
  parts = re.split('(\d+)', s)
../RetrieveDataProject/DownloadProjects/ChineseProjects/GPT-SoVITS/GPT_SoVITS/utils.py:258: SyntaxWarning: invalid escape sequence '\d'
  name_key = lambda _f: int(re.compile("._(\d+)\.pth").match(_f).group(1))
../RetrieveDataProject/DownloadProjects/ChineseProjects/GPT-SoVITS/GPT_SoVITS/text/english.py:233: SyntaxWarning: invalid escape sequence '\-'
  text = re.sub("[^ A-Za-z'.,?!\-]", "", text)
../RetrieveDataProject/DownloadProjects/ChineseProjects/talebook/tools/orzbook-spider.py:52: SyntaxWarning: invalid escape sequence '\)'
  passwd = re.findall(u'''提取码:([^\)]*)''', rsp.text)
../RetrieveDataProject/DownloadProjects/ChineseProjects/talebook/tools/tools

And English-speaking Programmers

In [32]:
chinese_root_directory = '../RetrieveDataProject/DownloadProjects/EnglishProjects'
for project_name in os.listdir(chinese_root_directory):
    project_path = os.path.join(chinese_root_directory, project_name)
    # walk through project path
    parse_all_python_files_in_project(project_path, project_name, author_type='English', db_conn=conn, db_table=table)

../RetrieveDataProject/DownloadProjects/EnglishProjects/langchain/docs/scripts/copy_templates.py:19: SyntaxWarning: invalid escape sequence '\!'
  content = re.sub("\!\[.*?\]\((.*?)\)", "", content)
../RetrieveDataProject/DownloadProjects/EnglishProjects/langchain/docs/scripts/copy_templates.py:33: SyntaxWarning: invalid escape sequence '\]'
  content = re.sub("\]\(\.\.\/", "](/docs/templates/", content)
../RetrieveDataProject/DownloadProjects/EnglishProjects/langchain/docs/scripts/resolve_local_links.py:14: SyntaxWarning: invalid escape sequence '\]'
  content = re.sub("\]\(\.\/", f"]({docs_link}", content)
../RetrieveDataProject/DownloadProjects/EnglishProjects/langchain/libs/experimental/langchain_experimental/plan_and_execute/planners/chat_planner.py:31: SyntaxWarning: invalid escape sequence '\s'
  steps = [Step(value=v) for v in re.split("\n\s*\d+\. ", text)[1:]]
../RetrieveDataProject/DownloadProjects/EnglishProjects/langchain/libs/experimental/langchain_experimental/generative_

Syntax error in file '../RetrieveDataProject/DownloadProjects/EnglishProjects/langchain/libs/langchain/tests/integration_tests/examples/non-utf8-encoding.py'
Syntax error in file '../RetrieveDataProject/DownloadProjects/EnglishProjects/langchain/libs/community/tests/integration_tests/examples/non-utf8-encoding.py'
Syntax error in file '../RetrieveDataProject/DownloadProjects/EnglishProjects/langchain/libs/community/tests/examples/non-utf8-encoding.py'


../RetrieveDataProject/DownloadProjects/EnglishProjects/langchain/libs/community/langchain_community/agent_toolkits/openapi/planner.py:321: SyntaxWarning: invalid escape sequence '\{'
  regex_name = re.compile(re.sub("\{.*?\}", ".*", name))
../RetrieveDataProject/DownloadProjects/EnglishProjects/langchain/templates/rag-semi-structured/rag_semi_structured/chain.py:47: SyntaxWarning: invalid escape sequence '\ '
  prompt_text = """You are an assistant tasked with summarizing tables and text. \
../RetrieveDataProject/DownloadProjects/EnglishProjects/langchain/templates/anthropic-iterative-search/anthropic_iterative_search/output_parser.py:9: SyntaxWarning: invalid escape sequence '\s'
  ext_list = re.findall(f"<{tag}\s?>(.+?)</{tag}\s?>", string, re.DOTALL)
../RetrieveDataProject/DownloadProjects/EnglishProjects/langchain/templates/anthropic-iterative-search/anthropic_iterative_search/output_parser.py:9: SyntaxWarning: invalid escape sequence '\s'
  ext_list = re.findall(f"<{tag}\s?>(.+?)

Syntax error in file '../RetrieveDataProject/DownloadProjects/EnglishProjects/models/research/cognitive_planning/train_supervised_active_vision.py'
Syntax error in file '../RetrieveDataProject/DownloadProjects/EnglishProjects/models/research/cognitive_planning/viz_active_vision_dataset_main.py'
Syntax error in file '../RetrieveDataProject/DownloadProjects/EnglishProjects/models/research/cognitive_planning/envs/active_vision_dataset_env.py'


../RetrieveDataProject/DownloadProjects/EnglishProjects/tacticalrmm/api/tacticalrmm/tacticalrmm/demo_data.py:299: SyntaxWarning: invalid escape sequence '\W'
  spooler_stdout = """
../RetrieveDataProject/DownloadProjects/EnglishProjects/pytorch/test/dynamo/test_misc.py:9613: SyntaxWarning: invalid escape sequence '\.'
  "^This compiled backward function is being run with torch\.use_deterministic_algorithms",
../RetrieveDataProject/DownloadProjects/EnglishProjects/pytorch/test/export/test_export.py:2554: SyntaxWarning: invalid escape sequence '\['
  RuntimeError, "shape\[0\] to be equal to 5, but got 7"
../RetrieveDataProject/DownloadProjects/EnglishProjects/pytorch/test/export/test_export.py:2975: SyntaxWarning: invalid escape sequence '\['
  RuntimeError, "shape\[0\] to be equal to 4, but got 7"
../RetrieveDataProject/DownloadProjects/EnglishProjects/pytorch/test/export/test_export.py:3056: SyntaxWarning: invalid escape sequence '\['
  with self.assertRaisesRegex(RuntimeError, "shape\

Syntax error in file '../RetrieveDataProject/DownloadProjects/EnglishProjects/django/tests/test_runner_apps/tagged/tests_syntax_error.py'


Let's see how does our data looks

In [33]:
conn = sqlite3.connect('data.db')
query = f"SELECT * FROM {table}"
df = pd.read_sql_query(query, conn)
df

,id,name,nameType,nameScope,author,project_name
0,0,sovits_path,variable,GlobalScope,Chinese,GPT-SoVITS
1,1,gpt_path,variable,GlobalScope,Chinese,GPT-SoVITS
2,2,is_half_str,variable,GlobalScope,Chinese,GPT-SoVITS
3,3,is_half,variable,GlobalScope,Chinese,GPT-SoVITS
4,4,is_share_str,variable,GlobalScope,Chinese,GPT-SoVITS
...,...,...,...,...,...,...
839640,839640,next_function,variable,FunctionScope,English,django
839641,839641,model_class,variable,FunctionScope,English,django
839642,839642,app_config,variable,FunctionScope,English,django
839643,839643,app_config,variable,FunctionScope,English,django


In [34]:
conn.close()